<center><font color="green" size="6"> <b>Sugarcane Yield Prediction using RS and ML<b> </font></center>
<center><img src="https://www.omex.com/wp-content/uploads/2020/10/Sugar-Cane-Harvest-1536x772.jpg" height="150"></center>

---

_Script elaborated by **Raul Roberto Poppiel**_ ([raulpoppiel@gmail.com](raulpoppiel@gmail.com)) for the [FAPESP](https://fapesp.br/en) project Nº [23/01062-1](https://bv.fapesp.br/en/bolsas/207973/satellite-imagery-and-machine-learning-for-sugarcane-yield-estimation-in-regions-of-sao-paulo-state/), led by Professor Ana Claudia dos Santos Luciano (analuciano@usp.br) from ESALQ/USP, Brazil. The methodology employed was developed by Rafaella Pironato Amaro ([rafaellapironato.amaro@gmail.com](rafaellapironato.amaro@gmail.com)) and is detailed in the document titled [Estimativa de produtividade da cana-de-açúcar a partir de imagens do satélite Sentinel-2A e o algoritmo de aprendizagem de máquina Random Forest](https://doi.org/10.11606/D.11.2023.tde-02102023-163947).

### Install and import tools

Geospatial modules

In [ ]:
# Load modules
import ee
import geemap

print('Modules loaded')

Modules loaded


Data science modules

In [ ]:
# Load modules
import os
import pandas as pd
import numpy as np
import time
from pathlib import Path

print('Modules loaded')

Modules loaded


### Connect to GEE and GDrive

In [ ]:
# GEE Authentication
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lGTJDf2NaVxcOaGW3VdSei4m8SJBd44QOs9nF_6gSkg&tc=TIkoTTufPIQVAW05cueB_EQeJVQVngTuLl5-RVg8Spc&cc=XfeSMAWJNn6STJQN8MEDrSYBcdGTkJV8HuCisggXaaE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXleGACjn6xzgNj4p1Tr-gtPrQjbNiAE9ksgzjFxf1CpqtvchYqXEHI

Successfully saved authorization token.


In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set GDrive paths

Define the output folders in GD

In [ ]:
# Define folder names
folder_name_root = 'Colab Notebooks'
folder_name_project = '01_SugarcaneYieldPrediction'
folder_name_farm = 'usinas_all'
folder_name_specific = '05_sentinel2_data_monthly'

# define your GEE username
username_gee = 'raulrpoppiel'

# Define sugarcane crop-season (SAFRA)
CropSeason = 1920 # 1920, 2021, 2122, 2223

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}'
project_path = f'{root_path}/{folder_name_project}'
farm_path = f'{project_path}/{folder_name_farm}'
out_path = f'{farm_path}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully.")
else:
  print("Output directory already exists.")

os.chdir(out_path)
print(os.getcwd(),'\n')  # Print the current working directory
pd.DataFrame(os.listdir(), columns=['List files'])  # List files and directories in the current directory

Output directory already exists.
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly 

List of files:
                                                Files
0  01_sentinel2_data_monthly_MEAN_safra_1920_merg...
1  01_sentinel2_data_monthly_MEAN_safra_2122_merg...
2  01_sentinel2_data_monthly_MEAN_safra_2223_merg...
3  01_sentinel2_data_monthly_MEAN_safra_2021_merg...


## Project-specific settings

### Feature collection

Example of attributes table structure:
```
 'ID_SIG': 'X19200890000003600027002', 🟢
 'AREA': 46.0963242244,
 'BLOCO': 2,
 'CODFAZ': 36,
 'COD_USINA': 89,
 'EST_Corte': 1, 🟢
 'Local': 'Usina 3',
 'Ordem': 'Latossolos',
 'POL': 0,
 'SAFRA': '1920', 🟢
 'SAFRA_real': 1819, 🟢
 'TAH': 0,
 'TALHAO': 17002,
 'TCH_ANT': 88.875, 🟢
 'Unidade_So': 'LV21',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',🟢
 'relevo': 'Suave Ondulado',
 'soloGeral': 'LV',
 'usina': 'M3'
```

Import the features from your Assets

In [ ]:
# Import polygons of agricultural plots (shapefiles) from your GEE

fc1 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_1')
# fc2 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_2')
# fc3 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_3')
# fc4 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_4')
# fc5 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_5')
# fc6 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_6')
# fc7 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_7')
# fc8 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_8')
# fc9 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_9')
# fc10 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_10')
# fc11 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_11')
# fc12 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_12')
# fc13 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_13')
# fc14 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_14')
# fc15 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_15')

print('FC1 size: ',fc1.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('FC1 attribute table (1st feature): \n')
display(fc1.limit(1).getInfo()['features'][0]['properties'])

FC1 size:  416 

FC1 attribute table (1st feature): 



{'AREA': 33.2284531537,
 'BLOCO': '0',
 'CODFAZ': '39',
 'COD_USINA': '28',
 'EST_Corte': '2',
 'ID_SIG': 'X20210280001003900000001',
 'Local': 'Usina 1',
 'Ordem': 'Argissolos',
 'POL': '0',
 'SAFRA': 2021,
 'SAFRA_real': '1920',
 'TAH': '0',
 'TALHAO': '1',
 'TCH_ANT': '70',
 'Unidade_So': 'PVA5',
 'VAR': 'RB867515',
 'VARIEDADE': 'RB867515',
 'relevo': 'Ondulado',
 'soloGeral': 'PV',
 'usina': 'M1'}

In [ ]:
# Select an attribute name (column) that contains the ID of polygons
ID = 'ID_SIG'

fc_to_reduction1 = fc1.select(ID)
# fc_to_reduction2 = fc2.select(ID)
# fc_to_reduction3 = fc3.select(ID)
# fc_to_reduction4 = fc4.select(ID)
# fc_to_reduction5 = fc5.select(ID)
# fc_to_reduction6 = fc6.select(ID)
# fc_to_reduction7 = fc7.select(ID)
# fc_to_reduction8 = fc8.select(ID)
# fc_to_reduction9 = fc9.select(ID)
# fc_to_reduction10 = fc10.select(ID)
# fc_to_reduction11 = fc11.select(ID)
# fc_to_reduction12 = fc12.select(ID)
# fc_to_reduction13 = fc13.select(ID)
# fc_to_reduction14 = fc14.select(ID)
# fc_to_reduction15 = fc15.select(ID)

In [ ]:
# Make a colection of collecions of geometries
SugarcaneData = ee.FeatureCollection([fc1
                                      # fc2,fc3,fc4,fc5,fc6,fc7,fc8,fc9,fc10,fc11,fc12,fc13,fc14,fc15
                                      ]).flatten()

# Display the number os polygons
print('FC size: ',SugarcaneData.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('Attribute table structure (1st feature): \n')
display(SugarcaneData.limit(1).getInfo()['features'][0]['properties'])

FC size:  6252 

Attribute table structure (1st feature): 



{'AREA': 33.2284531537,
 'BLOCO': '0',
 'CODFAZ': '39',
 'COD_USINA': '28',
 'EST_Corte': '2',
 'ID_SIG': 'X20210280001003900000001',
 'Local': 'Usina 1',
 'Ordem': 'Argissolos',
 'POL': '0',
 'SAFRA': 2021,
 'SAFRA_real': '1920',
 'TAH': '0',
 'TALHAO': '1',
 'TCH_ANT': '70',
 'Unidade_So': 'PVA5',
 'VAR': 'RB867515',
 'VARIEDADE': 'RB867515',
 'relevo': 'Ondulado',
 'soloGeral': 'PV',
 'usina': 'M1'}

### Crop-season and real period

In [ ]:
# Define the real period in years
if CropSeason == 1920:
    SrtYear = '2018'
    EndYear = '2019'
elif CropSeason == 2021:
    SrtYear = '2019'
    EndYear = '2020'
elif CropSeason == 2122:
    SrtYear = '2020'
    EndYear = '2021'
elif CropSeason == 2223:
    SrtYear = '2021'
    EndYear = '2022'
elif CropSeason == 2324:
    SrtYear = '2022'
    EndYear = '2023'
else:
    # Handle unknown CropSeason values
    print("Unknown CropSeason")

# Example:
# SAFRA_real = SAFRA - 1
SrtYear_real = str(int(SrtYear) - 1)
EndYear_real = str(int(EndYear) - 1)

# Display the results
print(f"Start: 01/04/{SrtYear} --> Real: 01/04/{SrtYear_real}")
print(f"End: 31/03/{EndYear} --> Real: 31/03/{EndYear_real}")

Start: 01/04/2019 --> Real: 01/04/2018
End: 31/03/2020 --> Real: 31/03/2019


In [ ]:
# Use real years for RS data acquisition (DO NOT change it)

# Montly (for using as covariates/predictors)
SrtDate = SrtYear+"-12-01"
EndDate = EndYear+"-03-31"

### Filter Sugarcane field data and bounding box

In [ ]:
# Compute bounding box for the fc
buffer_size = 5000; # define a value in meters

bbox_rect = SugarcaneData.geometry().bounds().buffer(buffer_size).bounds().getInfo()
bbox_coords = bbox_rect.get('coordinates')[0]
bbox = ee.Geometry.Rectangle([bbox_coords[0][0],bbox_coords[0][1],bbox_coords[2][0],bbox_coords[2][1]],None,False)

Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

Map.addLayer(SugarcaneData,{'color': 'FF0000'},'fc')
Map.addLayer(bbox, {'color': '#000060'}, 'Bbox')
Map.centerObject(SugarcaneData, 8)

Map.setControlVisibility()
Map

Map(center=[-21.420329614075595, -49.48433147616699], controls=(WidgetControl(options=['position', 'transparen…

## Monthly: manually agreggation

### Helper function

In [ ]:
# Spectral bands of Sentinel-2 satellite
S2_BANDS = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'] # Sentinel 2
STD_NAMES = ['blue','green','red','rededge1','rededge2','rededge3','nir','nir8A','swir1','swir2'] # standardize band names

# Select and rename spectral bands function
def selecBands(image):
  image =  image.select(S2_BANDS, STD_NAMES)
  return image;

# Custom cloud mask function (this is a strong cloud-shadow mask)
def cloudMask(image):
  BCloud = image.expression("b('B1') > 0 && b('B1') < 1000 && b('B2') > 0 && b('B2') < 1500").rename('BCloud')
  cirrus = image.expression("((((b('B1') * 2) + b('B2'))) / b('B3'))").rename('CIRRUS')
  clearMask = image.expression('BCloud > 0 &&' + 'CIRRUS < 2.4', {'CIRRUS': cirrus.select('CIRRUS'),'BCloud': BCloud.select('BCloud')})
  return image.updateMask(clearMask)

# Water mask
def waterMask(image):
  hansen_2020 = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').select('datamask');
  hansen_2020_wbodies = hansen_2020.neq(1).eq(0);
  return image.updateMask(hansen_2020_wbodies);

# Spectral indices function
def addIndices(image):
  idate = image.get('system:time_start');
  image = image.divide(10000)
  img = image.normalizedDifference(['nir','red']).rename('NDVI') # NDVI
  img = img.addBands(image.expression("2.5*((b('nir')-b('red'))/(b('nir')+(6*b('red'))-(7.5*(b('blue')))+1))").rename('EVI')) #EVI
  img = img.addBands(image.expression("1.5*(b('nir')-b('red'))/(b('nir')+b('red')+0.5)").rename('SAVI'))
  img = img.addBands(image.normalizedDifference(['swir1','nir']).rename('NDMI')) #LSWI = NDMI
  img = img.addBands(image.normalizedDifference(['green','nir']).rename('NDWI1')) #NDWI1
  img = img.addBands(image.normalizedDifference(['swir1','red']).rename('NDWI2')) #NDWI2
  img = img.addBands(image.normalizedDifference(['nir','rededge1']).rename('NDVIre1')) #NDVIRE
  img = img.addBands(image.normalizedDifference(['nir','rededge2']).rename('NDVIre2')) #NDVIRE
  img = img.addBands(image.normalizedDifference(['nir','rededge3']).rename('NDVIre3')) #NDVIRE

  img = img.addBands(image.expression("(b('nir')/b('rededge1'))-1").rename('CIre1'))#CIre
  img = img.addBands(image.expression("(b('nir')/b('rededge2'))-1").rename('CIre2'))#CIre
  img = img.addBands(image.expression("(b('nir')/b('rededge3'))-1").rename('CIre3'))#CIre

  img = img.addBands(image.expression("((b('nir')/b('rededge1'))-1)/(sqrt((b('nir')/b('rededge1'))+1))").rename('MSRre1'))
  img = img.addBands(image.expression("((b('nir')/b('rededge2'))-1)/(sqrt((b('nir')/b('rededge2'))+1))").rename('MSRre2'))
  img = img.addBands(image.expression("((b('nir')/b('rededge3'))-1)/(sqrt((b('nir')/b('rededge3'))+1))").rename('MSRre3'))
  img = img.addBands(image.normalizedDifference(['nir8A','swir1']).rename('SIWSI'))

  img = img.addBands(image.expression("(b('red'))").divide(6.553500175476074).rename('red'))
  img = img.addBands(image.expression("(b('nir'))").divide(6.553500175476074).rename('nir'))
  img = img.addBands(image.expression("(b('swir1'))").divide(6.553500175476074).rename('swir1'))
  img = img.addBands(image.expression("(b('swir2'))").divide(6.553500175476074).rename('swir2'))
  img = img.addBands(image.expression("(b('rededge1'))").divide(6.553500175476074).rename('rededge1'))
  img = img.addBands(image.expression("(b('rededge2'))").divide(6.553500175476074).rename('rededge2'))
  img = img.addBands(image.expression("(b('rededge3'))").divide(6.553500175476074).rename('rededge3'))
  img = img.addBands(image.expression("(b('nir8A'))").divide(6.553500175476074).rename('nir8A'))
  #result is the new image of all indices => get rid of reflectance bands
  return img.set({"system:time_start": ee.Number(idate)});

### Load Image Collection

Sentinel-2 MSI: MultiSpectral Instrument, Level-2A

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

In [ ]:
# Import collection and map helper functions
collectionS2 = (ee.ImageCollection("COPERNICUS/S2_SR")
                .filterBounds(bbox)
                .filterDate(SrtDate, EndDate)
                .map(cloudMask)
                .map(waterMask)
                .map(selecBands)
                .map(addIndices))

print('Collection size: ',str(collectionS2.size().getInfo())+' scenes \n');

Collection size:  588 scenes 



In [ ]:
# Get the most recent image
S2mostRecent = collectionS2.limit(10).sort('system:time_start', False).mean()
#display(pd.to_datetime(S2mostRecent.get('system:time_start').getInfo(), unit="ms"))
#print('Scene: ',S2mostRecent.get('system:index').getInfo())
print('Band names:', S2mostRecent.bandNames().getInfo())

# Get values from image
reducers = ee.Reducer.percentile([1,98], ['min','max']);
stats = S2mostRecent.select(['swir1', 'nir', 'red']).reduceRegion(reducer = reducers,
                              geometry = bbox,
                              scale = 5000,
                              tileScale = 16);

# Display the image
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

rgbFalseColor = {'bands': ['swir1', 'nir', 'red'],
                 'min': [stats.get('swir1_min'),stats.get('nir_min'),stats.get('red_min')],
                 'max': [stats.get('swir1_max'),stats.get('nir_max'),stats.get('red_max')],
                 'gamma': 1};

Map.addLayer(S2mostRecent, rgbFalseColor,'Most Recent Image - RGB(swir1-nir-red');
Map.centerObject(bbox, 8);

Map.setControlVisibility()
Map

Band names: ['NDVI', 'EVI', 'SAVI', 'NDMI', 'NDWI1', 'NDWI2', 'NDVIre1', 'NDVIre2', 'NDVIre3', 'CIre1', 'CIre2', 'CIre3', 'MSRre1', 'MSRre2', 'MSRre3', 'SIWSI', 'red', 'nir', 'swir1', 'swir2', 'rededge1', 'rededge2', 'rededge3', 'nir8A']


Map(center=[-21.406251078337068, -49.31341705029615], controls=(WidgetControl(options=['position', 'transparen…

### Temporal reduction (mosaics)

In [ ]:
# Define the way of aggregation
interval = 1  # time range for agregation (e.g., every 1 month)
intervalUnit = 'month'  # unit of time (e.g. 'year', 'month', 'day')
temporalReducer = ee.Reducer.median()  # how to reduce images in time range

In [ ]:
# Compute the intervalCount based on the provided start and end dates
intervalCount = ee.Date(EndDate).difference(ee.Date(SrtDate), intervalUnit).divide(interval).ceil().toInt()

# Get time window index sequence.
intervals = ee.List.sequence(0, intervalCount.subtract(1), 1)

# Function for temporal aggregation of images.
def reduceTemporal(i):
    # Calculate temporal composite.
    startRange = ee.Date(SrtDate).advance(ee.Number(interval).multiply(i), intervalUnit)
    endRange = startRange.advance(interval, intervalUnit)
    bandNames = collectionS2.first().bandNames().getInfo()
    temporalStat = collectionS2.filterDate(startRange, endRange).reduce(temporalReducer).rename(bandNames)

    # Set start date as a property.
    composite = temporalStat.set('system:index', ee.Date(startRange).format('YYYYMM'))
    composite = composite.set('system:time_start', ee.Date(startRange).format('YYYY-MM-dd'))

    # Return the composite image.
    return composite

# Map reductions over index sequence to calculate statistics for each interval.
reducedCollection_S2 = ee.ImageCollection(intervals.map(reduceTemporal))

In [ ]:
print('The collection size has been reduced to',reducedCollection_S2.size().getInfo(),f'images every {interval} {intervalUnit}')

display('From',reducedCollection_S2.first().get('system:time_start'))
display('To',reducedCollection_S2.sort('system:time_start',opt_ascending=False).first().get('system:time_start'))

The collection size has been reduced to 4 images every 1 month


'From'

'To'

In [ ]:
# Get the most recent image from monthly mosaics
MonthlyMosaics_mostRecent = reducedCollection_S2.first()
print('Month date:',MonthlyMosaics_mostRecent.get('system:index').getInfo(),'\n')
print('Band names:', MonthlyMosaics_mostRecent.bandNames().getInfo())

# Display the most recent image from the collection
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

# Get min/max values for the spectral band
rgbFalseColor = {'bands': ['swir1', 'nir', 'red'],
                 'min': [stats.get('swir1_min'),stats.get('nir_min'),stats.get('red_min')],
                 'max': [stats.get('swir1_max'),stats.get('nir_max'),stats.get('red_max')],
                 'gamma': 1}

ndviVis = {'bands':['NDVI'], 'min':0, 'max':1,'palette':['blue', 'white', 'green']}

Map.addLayer(MonthlyMosaics_mostRecent, rgbFalseColor,'Most Recent Mosaic - RGB(swir1-nir-red', False)
Map.addLayer(MonthlyMosaics_mostRecent, ndviVis,'Most Recent Mosaic - NDVI_median')
Map.centerObject(bbox, 8)

Map.setControlVisibility()
Map

Month date: 201912 

Band names: ['NDVI', 'EVI', 'SAVI', 'NDMI', 'NDWI1', 'NDWI2', 'NDVIre1', 'NDVIre2', 'NDVIre3', 'CIre1', 'CIre2', 'CIre3', 'MSRre1', 'MSRre2', 'MSRre3', 'SIWSI', 'red', 'nir', 'swir1', 'swir2', 'rededge1', 'rededge2', 'rededge3', 'nir8A']


Map(center=[-21.406251078337068, -49.31341705029615], controls=(WidgetControl(options=['position', 'transparen…

### Reduce by region and export as tables

In [ ]:
# Define the statistic
# "MEAN", "MAXIMUM", "MEDIAN","MINIMUM","MODE","STD","MIN_MAX","SUM","VARIANCE", "COUNT"
statistics_Type = 'MEAN'

# Define the output path to save results
global_stats_path1 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_1.csv')
# global_stats_path2 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_2.csv')
# global_stats_path3 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_3.csv')
# global_stats_path4 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_4.csv')
# global_stats_path5 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_5.csv')
# global_stats_path6 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_6.csv')
# global_stats_path7 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_7.csv')
# global_stats_path8 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_8.csv')
# global_stats_path9 =  os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_9.csv')
# global_stats_path10 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_10.csv')
# global_stats_path11 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_11.csv')
# global_stats_path12 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_12.csv')
# global_stats_path13 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_13.csv')
# global_stats_path14 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_14.csv')
# global_stats_path15 = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_15.csv')

print(global_stats_path1)
# print(global_stats_path2)
# print(global_stats_path3)
# print(global_stats_path4)
# print(global_stats_path5)
# print(global_stats_path6)
# print(global_stats_path7)
# print(global_stats_path8)
# print(global_stats_path9)
# print(global_stats_path10)
# print(global_stats_path11)
# print(global_stats_path12)
# print(global_stats_path13)
# print(global_stats_path14)
# print(global_stats_path15)

/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_1.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_2.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_3.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_4.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_5.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_6.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_

In [ ]:
# Reduzing daily climatic values by zone
scale = 50.0

parameters = {
    'in_value_raster': reducedCollection_S2,
    'statistics_type':statistics_Type,
    'scale':float(scale), # a high scale avoid missing values
    'tile_scale':16.0,
    'return_fc':False,
    'timeout':3000
    }

In [ ]:
geemap.zonal_statistics(in_zone_vector=fc_to_reduction1,out_file_path=global_stats_path1,**parameters)
time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction2,out_file_path=global_stats_path2,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction3,out_file_path=global_stats_path3,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_1.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_3.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction4,out_file_path=global_stats_path4,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction5,out_file_path=global_stats_path5,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction6,out_file_path=global_stats_path6,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_4.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_5.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_6.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction7,out_file_path=global_stats_path7,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction8,out_file_path=global_stats_path8,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_7.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_8.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction9,out_file_path=global_stats_path9,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction10,out_file_path=global_stats_path10,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_9.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_10.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction11,out_file_path=global_stats_path11,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction12,out_file_path=global_stats_path12,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction13,out_file_path=global_stats_path13,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_11.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_12.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_13.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction14,out_file_path=global_stats_path14,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction15,out_file_path=global_stats_path15,**parameters)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_14.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_15.csv


In [ ]:
time.sleep(10)

### Merge tables into a single one

In [ ]:
# Define the paths to your CSV files
file_paths = [
    os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_{i}.csv')
    for i in range(1, 16)
]

print(len(file_paths), 'paths')

15 paths


In [ ]:
# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()
file_paths
# Loop through the CSV files and merge them
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Remove the "system:index" column if it exists
        if "system:index" in df.columns:
            df = df.drop(columns=["system:index"])

        # Merge the data into the merged_df
        if merged_df.empty:
            merged_df = df
        else:  # Fix the typo here (remove "ID")
            merged_df = pd.concat([merged_df, df], axis=0, ignore_index=True)

# Move the last column (ID_SIG) to the first position
merged_df = merged_df[[merged_df.columns[-1]] + list(merged_df.columns[:-1])]
print('Dataframes merged!')

Dataframes merged!


### Imput missing values

In [ ]:
# Step 1: Group by variable (e.g., spectral indices)
df_grouped = merged_df.groupby(merged_df.columns.str[7:], axis=1)

# Step 2: Sort each group by the date obtained from the first 6 column header characters
sorted_groups = {}
for group_name, group_df in df_grouped:
    date_columns = sorted(group_df.columns, key=lambda x: x[:6])
    sorted_groups[group_name] = group_df[date_columns]

# Step 3: Impute missing data within each sorted group using bfill/ffill Methods
imputed_groups = {}
for group_name, group_df in sorted_groups.items():
    # Backfill missing values
    group_df = group_df.bfill(axis=1)
    # Forward fill missing values
    group_df = group_df.ffill(axis=1)
    imputed_groups[group_name] = group_df

# Convert the dictionary of DataFrames to a list of DataFrames
list_of_dfs = list(imputed_groups.values())

# Concatenate the DataFrames from the list
df_imputed = pd.concat(list_of_dfs, axis=1)

# Reset the index to ungroup the DataFrame
df_imputed = df_imputed.reset_index(drop=True)

# Display the imputed DataFrame
print(f"Missing values before imputing: {merged_df.isna().sum().sum()}")
print(f"Missing values after imputing:  {df_imputed.isna().sum().sum()}")
df_imputed

Missing values before imputing: 228624
Missing values after imputing:  0


,ID_SIG,202112_CIre1,202201_CIre1,202202_CIre1,202203_CIre1,202112_CIre2,202201_CIre2,202202_CIre2,202203_CIre2,202112_CIre3,...,202202_rededge3,202203_rededge3,202112_swir1,202201_swir1,202202_swir1,202203_swir1,202112_swir2,202201_swir2,202202_swir2,202203_swir2
0,X22230280001000100000030,0.531418,1.016546,1.016546,1.016546,0.115348,0.128831,0.128831,0.128831,0.014939,...,0.040281,0.040281,0.047979,0.038784,0.038784,0.038784,0.037535,0.028467,0.028467,0.028467
1,X22230280001001200000006,1.797414,3.270192,3.270192,3.270192,0.167085,0.235498,0.235498,0.235498,-0.003942,...,0.071424,0.071424,0.041094,0.032509,0.032509,0.032509,0.025208,0.016299,0.016299,0.016299
2,X22230280001001200000007,0.725467,1.331336,1.331336,1.331336,0.128636,0.142386,0.142386,0.142386,0.014202,...,0.047082,0.047082,0.049387,0.041698,0.041698,0.041698,0.037057,0.028791,0.028791,0.028791
3,X22230280001001200000011,1.143803,2.084713,2.084713,2.084713,0.139258,0.188411,0.188411,0.188411,0.007579,...,0.054735,0.054735,0.047193,0.036836,0.036836,0.036836,0.032458,0.021901,0.021901,0.021901
4,X22230280001001500000022,0.935352,1.811800,1.811800,1.811800,0.138599,0.191705,0.191705,0.191705,0.011531,...,0.050883,0.050883,0.045961,0.037828,0.037828,0.037828,0.034017,0.024866,0.024866,0.024866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4810,X22230890000016000018014,1.287922,1.482339,1.482339,1.482339,0.183682,0.175054,0.175054,0.175054,0.004382,...,0.045358,0.045358,0.045657,0.037630,0.037630,0.037630,0.031037,0.022006,0.022006,0.022006
4811,X22230890000016200017002,0.790123,1.158484,1.158484,1.158484,0.145277,0.117272,0.117272,0.117272,0.007275,...,0.058563,0.058563,0.057695,0.050761,0.050761,0.050761,0.042226,0.032230,0.032230,0.032230
4812,X22230890000016400017007,0.455319,0.700523,0.700523,0.700523,0.127825,0.118082,0.118082,0.118082,0.028897,...,0.045895,0.045895,0.073299,0.065032,0.065032,0.065032,0.055255,0.046167,0.046167,0.046167
4813,X22230890000016400017008,0.431374,0.547766,0.547766,0.547766,0.118031,0.103734,0.103734,0.103734,0.026715,...,0.046352,0.046352,0.072277,0.067659,0.067659,0.067659,0.056423,0.050605,0.050605,0.050605


### Export results as table

In [ ]:
# Define the path for the output merged CSV file
merged_csv_path = os.path.join(out_path, f'01_sentinel2_data_monthly_{statistics_Type}_safra_{CropSeason}_merged.csv')

# Save the merged DataFrame to a CSV file
df_imputed.to_csv(merged_csv_path, encoding='utf-8', index=False)

# Print the path to the merged CSV file
print(f'Merged CSV file saved at: {merged_csv_path}')

Merged CSV file saved at: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2223_merged.csv


In [ ]:
# Delete CSV files
for file_path in file_paths:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_1.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_2.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_3.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_4.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_5.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_2021_6.csv
Deleted: /conten